# Kamiwaza Doc Ingestion/Embedding/Vector searching

## Overview

In this we are:

1. Checking out the Huggingface Transformers docs from git to give us something to index
2. Adding that to the Kamiwaza catalog
3. Mass ingesting the content into Kamiwaza with the Kamiwaza embedding (SentenceTransformer) and vectordb (MilvusImplementation) middlewares, with Ray parallelizing
4. Doing vector queries against the code
5. Extra Credit: Deploy Kamiwaza Agent Zero https://github.com/kamiwaza-ai/agentzero and use the chat interface to chat with the docs

We are actually making this **much more complicated** than it needs to be in Kamiwaza. An example of a "simple" version that does the same thing (other than checkout out the docs we intend to index):

```
service = IngestionService('sec_data')
service.ingest_and_embed(
    's3://kamiwaza-demo/pdfs/parsed',
    description='SEC 10-K Filings',
    # we pass none because we are assuming the catalog has these already,
    # so this is just tagging the secrets to these catalog entries
    secret_access_key = {'cloudflare_secret_access_key': None},
    access_key_id = {'cloudflare_access_key_id': None},
    endpoint_url = {'cloudflare_endpoint': None},
    region = {'cloudflare_region': None}
)
```

However, for the sake of showing more capabilities step-by-step, we are doing this a bit more step by step


In [ ]:
# First, go get the code - you need 'git' installed!

# This is about 200MiB of data; we are checking out only the /docs/source/en subfolder
!mkdir -p /var/tmp/transformers-docs
!cd /var/tmp/transformers-docs && git init transformers-docs && cd transformers-docs && git remote add -f origin https://github.com/huggingface/transformers.git
!cd /var/tmp/transformers-docs/transformers-docs && git config core.sparseCheckout true && echo "docs/source/en/" >> .git/info/sparse-checkout && git fetch --depth 1 origin main && git checkout main 2>/dev/null

In [ ]:
# just various imports; some of these are used later, but we import them all now so our
# logger adjustments can affect them
from kamiwaza.services.catalog.services import DataService
from kamiwaza.services.retrieval.runners.file import FileRunner
from kamiwaza.middleware.embedding.sentencetransformers import SentenceTransformerEmbedding
from kamiwaza.middleware.vectordb.milvus import MilvusImplementation
from typing import List, Dict, Any

import logging
logging.basicConfig(level=logging.WARNING)

# be less noisy
logging.getLogger("requests").setLevel(logging.WARNING)
for logger_name in logging.root.manager.loggerDict:
    logger = logging.getLogger(logger_name)
    logger.setLevel(logging.WARNING)

# and test our venv
import sys
import ray

# This will print the path to the Python interpreter being used,
# which usually includes the name of the virtual environment
print("Current Virtual Environment:", sys.executable)
default_model_name=SentenceTransformerEmbedding().model_name
print(f"Default embedding model: {default_model_name}")

DOCS_PATH='/var/tmp/transformers-docs'



### Suppress verbose logging

In [ ]:
# This cell is basically 100% just to suppress noisy debug messages
# This should work across all nodes in a cluster when using Ray

from typing import List
from ray.util.scheduling_strategies import NodeAffinitySchedulingStrategy

# For Jupyter, dispense with the super-verbose debugging
def set_logging_level(level=logging.WARNING):
    logging.getLogger().setLevel(level)

#### Ray Init - None of this is functionally required; Ray will turn on automatically if
#### needed when called; however, we explicitly initialize it here to lower logging to
#### the verbose logging, as notebook settings will not carry into ray since those
#### environments are totally encapsulated (even if local)
if not ray.is_initialized():
    # Example of manually setting required PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION
    # not needed if you started ray with Kamiwaza start-ray.sh but if you have
    # an existing or self-managed ray cluster, this may be needed
    runtime_env = {
        "env_vars": {
            "PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION": "python"
        }
    }
    ray.init(configure_logging=True, logging_level=logging.ERROR, runtime_env=runtime_env)

@ray.remote
def init_worker():
    set_logging_level()

@ray.remote
def node_affinity_func():
    return ray.get_runtime_context().get_node_id()

# Get list of nodes
nodes: List[str] = ray.nodes()

# Run init_worker on each node
for node in nodes:
    node_id = node.get('NodeID')
    init_worker.options(
        scheduling_strategy=NodeAffinitySchedulingStrategy(
            node_id=node_id,
            soft=False,
        )
    ).remote()

# Set the level for all loggers to WARNING
for logger_name in logging.root.manager.loggerDict:
    logger = logging.getLogger(logger_name)
    logger.setLevel(logging.WARNING)

# Ingest, Embed, and Analyze Code

In this example we show how to:

- Ingest code by path
- Embed it
- Query it with the vector search
- Extra Credit - turn up the Kamiwaza GUI, deploy a model that understands code (we recommend DeepSeek-Coder-33B or Phind-CodeLlama-34B-V2) and ask questions!

  

# Step 0: Define Processing 

This is a very simple function to just filter on file type, turn the binary read stream to text, and then return it.

In another demo, we use langchain RecursiveCharacterTextSplitter with Language.PYTHON in a code example


In [ ]:

# We define ProcessCode as a function to be called in our ray pipeline later; all we are really doing is
# decoding because we are reading as binary (which delivers all the bytes in a batch in byte order;
# we can specify ray_mode='read_text' or any other ray.data.* function into our retriever though,
# but read_text won't guarantee us all the lines in a file in order
def ProcessDoc(chunk: ray.data.Dataset, catalog_urn: str = None) -> List[Dict[str, Any]]:
    # chunk has 'item' if binary, 'text' if text, 'path' unless over-ridden
    # other retrieval functions other than read_binary_data (default) and read_text may vary
    try:
        # an example of restricting types
        allowed_extensions = ['.py', '.html', '.yaml', '.toml', '.c', '.txt', '.md', '.json']
        if not any(chunk['path'].endswith(ext) for ext in allowed_extensions):
            return []

        try:
            # we are reading in binary
            text = chunk['bytes'].decode('utf-8')
        except Exception:
            # This likely won't happen with the extension but this try/except is useful
            # if you just want to ingest data and some is not necessarily text
            print(f"text in {chunk['path']+' ' if 'path' in chunk else ''}was not decodable")
            return []

        offset = 0  # default offset

        # Here we are going to do a "cool thing" Kamiwaza enables. Many/all hf md docs
        # start with a html-formatted copyright header <!-- stuff -->
        # That is a semantic distraction that is meaningless.
        # 1. this is why you should preprocess your docs :) (better than we are in this demo)
        # 2. Kamiwaza eanbles a trick; we can pass an offset to our embedder, and kamiwaza,
        #    when needing to split the chunk further for the model, will actually USE our
        #    offset as the base offset, instead of 0. so instead of the first chunk being, say
        #    byte 0-2200, if we say "offset 500", it will be 500-2700. 
        #
        # So we will use that trick now to get better semantic meaning from these docs
        # without modifying them (as that reflects the real world; these are git-tracked
        # so modifying them means you do it on the fly, which means you have to track
        # the "original" file's offsets; or you need a copy (boo!)
        #
        # So this code finds the first title, if it can, and assuming it's up to 20%
        # into the doc, strips it and sets the offset, it will pass that as the offset
        # and pass only the next after that offset!
        if chunk['path'].endswith('.md'):
            header_location = text.find('\n# ')
            # If the header is found and is less than 20% into the document
            if header_location != -1 and header_location <= len(text) * 0.2:
                # Ensure offset is the UTF-8 byte distance
                offset = len(text[:header_location].encode('utf-8'))
        
        # In some documents, we might want to return something like
        # chunk['preamble'] = f"File: {chunk['path']} <<<\n"
        # Kamiwaza's splitter will replicate a preamble, across all chunks. If you have chunks in order
        # where you specify "top-level <<< middle <<< lower\n" kamiwaza will both replicate the whole
        # preamble across text chunks for splitting, and merge chunks that have at least the top-level
        # the same, to maximize your embedder context (and will avoid merging chunks that cross
        # the top-level boundary) (And you can specify preamble_separator='@@@@@@' or whatever when calling 
        # kamiwaza.middleware.embedding.embedding_sentence_transformers.SentenceTransformerEmbedding.__call__() or chunk_text()
        chunks = [{
            'model_name': default_model_name,
            'source': chunk['path'] if 'path' in chunk else None,
            'text': text[header_location+1:] if offset else text,  # Drop everything before the first title
            'offset': offset,
            'catalog_urn': catalog_urn
        }]
        return chunks

    except:
        # If one of our sources doesn't read, we will just skip it
        logger.warning("We skipped a chunk: %s" % chunk['path'] if chunk and 'path' in chunk else 'Unknown')
        return []



### Step 1: Add the location of your data to the catalog. (In this case, it's just a path)

In [ ]:
COLLECTION_NAME='transformerdocs' # name for vectordb collection

In [ ]:
# This was already imported at the top, so we could set the logging level
#from kamiwaza.services.catalog.services import DataService

# In this case we go to out DataService, which powers the /catalog API endpoints among other
# things; it's fine to also instantiate the catalog plugin directly and use it
data_service = DataService('data_datahub') # not yet a default, but may become one

# Super simple catalog ingestion
catalog_items = data_service.create_dataset(
    dataset_name=DOCS_PATH,
    platform='file', # s3 is the other major platform, for any s3-compatible object; gcs and azure WIP
    description='Markdown documents for Huggingface transformers',
    additional_properties = {'collection': COLLECTION_NAME} # Kamiwaza ingesters can set this automatically, but we will set it here
)

print("Ingested")

### Step 2: Retrieve Datasets

In [ ]:
# moved these up top to silence debug loggings
#from kamiwaza.services.retrieval.runners import FileRunner
#from kamiwaza.middleware.embedding import SentenceTransformerEmbedding
#from kamiwaza.middleware.vectordb import MilvusImplementation


# we will retrieve the urn of the dataset
catalog_items


In [ ]:

# Give the dataset to the runner; given that this is file, we could iterate the datasets here and pull
# the path out, however, the runner can fetch via ray, meaning we have ray.data.Dataset objects
# we can then use in pipelines which will be automatically distributed
# We could pass query= here to the runner also, or a Pathspec (matching the datahub Pathspec), but
# since expect our dataset to be a directory and we are happy taking it and all its children, we will not
runner = FileRunner(catalog_items)
runner.logger.setLevel(logging.WARN)

# Definition of vectorDB Fields
# Kamiwaza, if you do not specify otherwise, will automatically have the following fields in the schema for a collection/db:
# 1. model_name: Absolutely critical because you need to embed vector queries with the same embedding model that you used; we will support model_ids in the future
# 2. source: for retrieval; this will generally be the file or object path
# 3. offset: this is the position in the file; many of the higher level kamiwaza services for manipulating data
#   can auto-track this info
# 4. catalog_urn: enabling rag; this allows Kamiwaza to automatically know how to track down data from the vectordb
#    (we likewise have several higher-order methods to ingest data into the catalog and vectordb where we automatically
#     store 'collection' as metadata)
fields = [
    #('model_name', str, default_model_name),
    #('source', str)
    #('offset', int),
    ('catalog_urn', str) # using Kamiwaza ingest services this is done for you
]

vectordb = MilvusImplementation()

# kamiwaza add scheme will call milvus.client.add_schema but normalize with an uuid index named id and an embedding of supplied dims
# if we were unsure of the dims, we can instantiate SentenceTransformers(model=MODEL_NAME) ahead of time and check the .dims property
# eg
# embedder = SentenceTransformers() #default is BAAI/llm-embedder at time of writing, likely to change
vectordb.drop_schema(COLLECTION_NAME)

# we could interrogate dims from our model if we didn't want to hardcode it
vectordb.add_schema(COLLECTION_NAME, dims=768, fieldlist=fields)


# Markdown - "normal" short form example

```python
# note: this is markdown
# in many of these things we can pass params
#   read_text totally optional here, default 'read_binary' - expanding tests here - https://docs.ray.io/en/latest/data/api/input_output.html
#   we pass **kwargs on, so you can specify any key=value argument to go to the ray call that is valid
# map_batches is a ray call and coordinates the handoff of batches from each dataset to the next, see https://docs.ray.io/en/master/data/api/doc/ray.data.Dataset.map_batches.html
# our SentenceTransformerEmbedding can take args but if you deviate from default, map_batches expects you to pass fn_args, fn_kwargs, fn_constructor_args, fn_constructor_kwargs
#     (see again https://docs.ray.io/en/master/data/api/doc/ray.data.Dataset.map_batches.html)
# Milvus and our other vectordbs can also take quite a few args, including host, port, u/p; it defaults to where your 'default' vectordb is instantiated

res = ds.flat_map(ProcessCode).map_batches(
    SentenceTransformerEmbedding,
    concurrency=10,
    num_gpus=0.1,
    batch_size=16384,
    zero_copy_batch=True,
).map_batches(
    MilvusImplementation,
    concurrency=10,
    num_gpus=0.1,
    batch_size=16384,
    zero_copy_batch=True,
    fn_kwargs={COLLECTION_NAME: COLLECTION_NAME}
).take_all()
```

### Runner

A runner is a helper for retrieval in Kamiwaza. It handles things like:

* Managing and proxying credentials for object if desired
* Converting bucket/directory catalog entries into full sets of retrievable files if recursive (default:True); you can specify a PathSpec to filter also
* Safely accessing local vs network files with correct paths
* Providing on-the-fly automatic decompression for compressed file types supported (gz, bz2, brotli, lz4, zstd, snappy, at this moment)
* Performing byte-ranged retrievals automatically for lists of results from vector searches that have offsets set
* Retrieving the data as ray.data datasets, allowing you to then use ray calls like .map_batches() as we do here to have massively parallel ingestion 

Like most things in the Kamiwaza stack, totally optional. Kamiwaza's retrieval service and ingest service both make use of runners to some extent; but to the point of this
notebook, you could retrieve bytes, rows, etc, in any manner you choose, and still pass them to Kamiwaza's embedding tools, vector tools, etc. You can also 
retrieve data however you choose at use ray data calls like ray.data.from_pandas() (or _numpy or _arrow) to still have a dataset and get the ray benefits.


In [ ]:
catalog_urn = runner.datasets if isinstance(runner.datasets, str) else runner.datasets[0]
catalog_urn

## The Pipeline

First, you don't **need** to read this. This is a lot of fun detail about some things under the hood so to speak. 

Here we:

1. use our runner to return the ray datasets
2. pass each dataset (which in this case, each file is a "dataset") into our ProcessCode function to get back the text with the metadata
3. feed that into kamiwaza SentenceTransformerEmbedding, one of our middlewares for embedding- see below for more detail
4. The next step of our pipeline feeds that into our Milvus vectordb middleware. More on that below also.

Some detail about the embedder here:

* in this case, we are using the default embedder (BAAI/llm-embedder at the moment)
* that actually both chunks the text and generates embeddings, parallelized by ray; we get back embedded batches
* by default Kamiwaza fills the model context window (512 tokens in this case) using "smart splitting" based on the size of the model.
* overlap is configurable as number of tokens (default 32)
* preambles are automatically accounted for and added to all chunks to tie their (usually significant) semantic context to later chunks of the document
* If you pass chunks with a 'source' (which we added in our ProcessCode() function based on the file name), Kamiwaza will automatically generate offsets, so long as the text does not have un-tokenizable parts (that is, sections where the tokenizer of the embedding model generates the `[UNK]` special token

Some details about the vectordb use here:

* We provide a middleware to handle some of the undifferentiated lifting for vectordbs, and simplify usage.
* We automatically create an id column, an embedding column, and we automatically add kamiwaza autofields (model_name, source, offset); you can override these using the autofields param, but we see this generally as critical (model_name to ensure you can switch embedders and still know which model to use to generate the query embedding; source to aid in retrieval, so the vector search returns where the embedding came from, and offset, so for larger files you can do byte-range reads; eg, the `AgentZero` chat interface, which integrates with Kamiwaza as a chatbot app, by default retrieves offset-500 to offset + 4096 characters
* We by default select some sane choices for the default use case (qa/retrieval); you can specify things like index parameters, nprobe/nlimit, etc, but we try to select reasonable out-of-the-box defaults **for that use case**; this is one of those places where if your use case is different you may quickly want to change those
* Like all Kamiwaza components, though, you don't need to use it; or you can use it in part, as MilvusImplementation.client is the pymilvus client instance


In [ ]:
# an "expanded" version of the example - this is code

# Note: if debugging is passing through you will quite likely see messages like this:
# (MapWorker(ReadBinary->FlatMap(ProcessCode)->MapBatches(SentenceTransformerEmbedding)) pid=8445) Token indices sequence length is longer than the specified maximum sequence length for this model (519 > 512). Running this sequence through the model will result in indexing errors
# 
# Kamiwaza intentionally tokenizes and splits without passing the unsplit tensors to the model
# you can set KAMIWAZA_DEBUG_MODE=1 in the environment and Kamiwaza will also test sequence length before encoding,
# and if a sequence is too long you will get a warning from Kamiwaza that begins
# "!!!!!!!!!!!!!!!!!!! WARNING: text [text] is [length], which is longer than the model's max_length of [max]"
# 

# in many of these things we can pass params
#   read_text totally optional here, default 'read_binary' - expanding tests here - https://docs.ray.io/en/latest/data/api/input_output.html
#   we pass **kwargs on, so you can specify any key=value argument to go to the ray call that is valid
# map_batches is a ray call and coordinates the handoff of batches from each dataset to the next, see https://docs.ray.io/en/master/data/api/doc/ray.data.Dataset.map_batches.html
# our SentenceTransformerEmbedding can take args but if you deviate from default, map_batches expects you to pass fn_args, fn_kwargs, fn_constructor_args, fn_constructor_kwargs
#     (see again https://docs.ray.io/en/master/data/api/doc/ray.data.Dataset.map_batches.html)
# Milvus and our other vectordbs can also take quite a few args, including host, port, u/p; it defaults to where your 'default' vectordb is instantiated

# Step 1: Get Ray dataset
ray_dataset = runner.get_ray_datasets()
print(f"Process {ray_dataset.count()} datasets")
import psutil

# Lots of ways to slice our resources but this should be reasonable
num_cpus = int(psutil.cpu_count() * 0.33)

# we are going to set a "free for all" level of gpu resource reservation
# but only if a gpu is detected
num_gpus = .01 if ray.available_resources().get('GPU', None) else None

# Step 2: Apply flat_map with ProcessCode function (pass in the catalog_urn to add it to the object)
processed_dataset = ray_dataset.flat_map(ProcessDoc, fn_args={'catalog_urn': catalog_urn})

# Step 3: feed to kamiwaza embedder middleware
embedded_dataset = processed_dataset.map_batches(
    SentenceTransformerEmbedding,
    concurrency=(1,num_cpus),
    num_gpus = num_gpus,
    batch_size=64, # relatively small batch as we are handing in batches of full files
    zero_copy_batch=True, # more performant if not mutating the input
)

# Step 4: Apply map_batches and push the embeddings into the vectordb
milvus_dataset = embedded_dataset.map_batches(
    MilvusImplementation,
    concurrency=(1,num_cpus),
    num_gpus = num_gpus,
    batch_size=128,
    zero_copy_batch=True,
    fn_kwargs={'collection_name': COLLECTION_NAME}
)

# note we apply the catalog_urn here we derived from above; the kamiwaza ingestion service will inject that automatically
# if we were using it rather than doing the step-by-step

# Step 5: Take all results (this materializes the pipeline, as would .materialize() - without that things would not begin to evaluate)
results = milvus_dataset.take_all()
len(results)


## Process complete

Let's discuss a few things we will typically see in the output here

* We will get a message reepatedly about `sequence length longer than the specified maximum` - that's normal. SentenceTransformers warns that when you tokenize too long of a string. Kamiwaza splits those tokens
* Some files will trigger `excessive special tokens on input string; disabling autooffset` or - this just means that the tokenizer sees text it cannot tokenize, and when that happens, Kamiwaza doesn't try to automatically generate offsets any more because it would be computationally difficult. You can do one of two things: 1. Chunk yourself and record the offset; 2. If the file isn't excessively large and is a strong result, retrieve all of it -- we are going to be adding another embedding abstraction in 0.3/0.4 which will allow us to capture offsets even on files with unknowns; this **is** a doable thing, even with SentenceTransformers
* We even get (perhaps) a small number of `!!!! on_final_token exceeded 20 - unable to align tokens - disabling autooffset` errors - again, this means we are going to not have offsets (at least past that point) for that given chunk (a file, in this case) - this is something we typically only see with things with extremely dense text, like yaml files that have base64-encoded certificates; when 20+ tokens represent unbroken text, it becomes excessive work to align that if the embedder is not supply it; so Kamiwaza gives up; again, in 0.3/0.4 we will address that using another embedder that returns offsets during the embedding process so we don't have to do that for it)

In [ ]:
results

## Now we have searchable vectors

In the examples below we do various vector searches. It's important to note that:

1. Dense embeddings aren't always the best option for retrieval; sparse embeddings (like SPLADE) or bag-or-words representations with reverse-index search like BM25 are also very valuable
2. An upcoming Kamiwaza release will automate hybrid vectors or hybrid vector+reverse index searching
3. Kamiwaza's retrieval engine will have an auto-reranked in the near future; that will allow you to retrieve a larger context of docs (say, 20 or 50 instead of 5 or 10) and allow Kamiwaza to then rerank the chunks specifically against the user query; this also works very well with BM25 and Sparse vectors
4. We've done ~nothing to optimize here. We could have:

* Stripped the redundant copyright notice
* Found the doc title and put it in the preamble
* Actually chunked on headers/subheaders (and added subheaders to the preamble)
* etc

Exercise left to the reader for now!

In [ ]:
# Simple search
# Note: this flow of 'embed the query and get back results' will be reduced to a single method in the future in kamiwaza middleware

st = SentenceTransformerEmbedding()
query = "Represent this query for retrieving relevant documents: what are the params for AutoModel.from_pretrained()?"
# For the record, in our tests, the 3rd result here (main_classes/model.md offset 1894) is what we
# imagined getting back; you'll also possibly see 2 results with the same path and offset 0;
# which means that there were two different embeddings from that file that were highly relevant, but
# the tokenizer had issues with some passages or kamiwaza had a problem aligning them (this is
# more likely with dense passages of code, as it is more likely to have large strings of '##suffix' tokens
# and with SentenceTransformers, we must align tokens and text ourselves

# usecase and mode allow Kamiwaza to map to model instructions if available
# some models, especially for query embeddings, expect an instruction like
# "Represent this passage for searching relevent texts: " etc
query_embedding = st.generate_embedding(text=query, usecase='qa', mode='query')
search_results = vectordb.search_vector(COLLECTION_NAME, query_embedding, output_fields=["*"])

for i in range(0,min(10,len(search_results[0])),1):
    print(f"{search_results[0].distances[i]} {search_results[0][i].entity.get('source')} offset {search_results[0][i].entity.get('offset')}")



In [ ]:
# Search of a specific fact - we use this in the AgentZero retrieval demo!

st = SentenceTransformerEmbedding()
query = "who wrote the paper that presented the MBart model??"

# as an aside, without our header-stripping in the ProcessDoc code, first chunk of the umt5.md doc
# is not semantically strong enough for the embedding model to correctly associate it with the query,
# despite the fact that the first real readable text under the title is
# "The UMT5 model was proposed in UniMax: Fairer and More Effective Language Sampling for Large-Scale Multilingual Pretraining by
# Hyung Won Chung, Xavier Garcia, Adam Roberts, Yi Tay, Orhan Firat, Sharan Narang, Noah Constant."

# Reams could be written on this in the micro and how it points toward hybrid searching, entity extraction, etc.

query_embedding = st.generate_embedding(text=query, usecase='qa', mode='query')
search_results = vectordb.search_vector(COLLECTION_NAME, query_embedding, output_fields=["*"])

for i in range(0,min(10,len(search_results[0])),1):
    print(f"{search_results[0].distances[i]} {search_results[0][i].entity.get('source')} offset {search_results[0][i].entity.get('offset')}")



In [ ]:
# verbatim search search
# here we query with an exact passage from the umt5.md page; this of course is what we'd expect to be the first result
# but shockingly similar how other embeddings are - although llm-embedder has a similarity distribution
# of [0.6,1] because of the contrastive learning approach;

st = SentenceTransformerEmbedding()

# Verbatim text selecting from from /model_doc/umt5.md
query = "As part of our contribution, we release: (i) an improved and refreshed mC4 multilingual corpus consisting of 29 trillion characters across 107 languages, and (ii) a suite of pretrained umT5 model checkpoints trained with UniMax sampling."

# usecase and mode allow Kmiwaza to map to model instructions if available
# some models, especially for query embeddings, expect an instruction like
# "Represent this passage for searching relevent texts: " etc
query_embedding = st.generate_embedding(text=query, usecase='qa', mode='query')
search_results = vectordb.search_vector(COLLECTION_NAME, query_embedding, output_fields=["*"])

for i in range(0,min(10,len(search_results[0])),1):
    print(f"{search_results[0].distances[i]} {search_results[0][i].entity.get('source')} offset {search_results[0][i].entity.get('offset')}")


In [ ]:
# unrelated search query; should return a lower relevance; probably < .8 with Milvus

# here we search something utterly unrelated; however, dense embeddings often have
# some random overlap between vectors; it can be from words, phrases, sentence
# structure

st = SentenceTransformerEmbedding()
query = "What movie paired Sandra Bullock and Wesley Snipes early in her career?"

# usecase and mode allow Kmiwaza to map to model instructions if available
# some models, especially for query embeddings, expect an instruction like
# "Represent this passage for searching relevent texts: " etc
query_embedding = st.generate_embedding(text=query, usecase='qa', mode='query')
search_results = vectordb.search_vector(COLLECTION_NAME, query_embedding, output_fields=["*"])

for i in range(0,min(10,len(search_results[0])),1):
    print(f"{search_results[0].distances[i]} {search_results[0][i].entity.get('source')} offset {search_results[0][i].entity.get('offset')}")
